# 6.1 텍스트 데이터 다루기

## 6.1.1 단어와 문자의 원-핫 인코딩

In [0]:
# 단어 수준의 원-핫 인코딩하기
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']  # 초기 데이터: 각원소가 샘플

token_index = {}  # 데이터에 있는 모든 토큰의 인덱스를 구축
for sample in samples:
  for word in sample.split():   # split() 메서드를 사용하여 샘플을 토큰으로 나눔
    if word not in token_index:
      token_index[word] = len(token_index) + 1  # 단어마다 고유한 인덱스를 할당합니다. 인덱스 0은 사용하지 않는다  

max_length = 10  # 샘플을 벡터로 변환합니다. 각 샘플에서 max_length까지 단어만 사용
results = np.zeros(shape=(len(samples),
                          max_length,
                          max(token_index.values()) + 1))   # 경과를 저장할 배열
for i, sample in enumerate(samples):
  for j, word in list(enumerate(sample.split()))[:max_length]:
    index = token_index.get(word)
    results[i, j, index] = 1.

In [0]:
# 문자 수준 원-핫 인코딩하기
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # 출력 가능한 모든 아스키 문자
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
  for j, character in enumerate(sample):
    index = token_index.get(character)
    results[i, j, index] = 1.

In [0]:
# 캐라스를 사용한 단어 수준의 원-핫 인코딩하기
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words=1000) # 가장 빈도가 높은 1000개의 단어만 선택하도록 Tokenizer 객체를 만듬
tokenizer.fit_on_texts(samples)  # 단어 인덱스를 구축

sequences = tokenizer.texts_to_sequences(samples)  # 문자열을 정수 인덱스의 리스트로 변환

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')  # 직접 원-핫 이진 벡터 표현을 얻을수 있음. 원-핫 인코딩 외에 다른 벡터화 방법들도 제공

word_index = tokenizer.word_index  # 계산된 단어 인덱스를 구함
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

9개의 고유한 토큰을 찾았습니다.


In [0]:
# 해싱 기법을 사용한 단어 수준의 원-핫 인코딩하기
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

dimensionality = 1000 # 단어를 크기가 1000인 벡터로 저장. 1000개의 단어가 있다면 해싱 충돌이 늘어나고 인코딩의 정확도가 감소될것
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
  for j, word in list(enumerate(sample.split()))[:max_length]:
    index = abs(hash(word)) % dimensionality # 단어를 해싱하여 0과 1000사이의 랜덤한 정수 인덱스로 변환
    results[i, j, index] = 1.

## 6.1.2 단어의 임베딩 사용하기

In [0]:
# Embedding 층의 객체 생성하기
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64)  # Embedding 층은 적어도 2개의 매개변수를 받음. 가능한 토큰의 개수(여기서는 1000으로 단어 인덱스 최댓값 +1)와 임베딩차원(여기서는 64)임

In [0]:
# Embedding 층에 사용할 IMDB 데이터 로드하기
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000  # 특성으로 사용할 단어의 수
maxlen = 20  # 사용할 텍스트의 길이(가장 빈번한 max_features개의 단어만 사용)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features) # 정수 리스트로 데이터를 로드

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)  # 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

17465344/17464789 [==============================] - 0s 0us/step


In [0]:
# IMDB 데이터에 Embedding 층과 분류기 사용하기
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))  # 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 Embedding층에 input_length를 지정. Embedding 층의 출력 크기는(samples, maxlen, 8)이 됨

model.add(Flatten())  # 3D 임베딩 텐서를 (samples, maxlen*8) 크기의 2D 텐서로 펼침

model.add(Dense(1, activation='sigmoid'))  # 분류기를 추가
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 74us/step - loss: 0.6622 - acc: 0.6344 - val_loss: 0.6009 - val_acc: 0.7126
Epoch 2/10
20000/20000 [==============================] - 1s 57us/step - loss: 0.5275 - acc: 0.7532 - val_loss: 0.5157 - val_acc: 0.7340
Epoch 3/10
20000/20000 [==============================] - 1s 61us/step - loss: 0.4555 - acc: 0.7886 - val_loss: 0.4948 - val_acc: 0.7504
Epoch 4/10
20000/20000 [==============================] - 1s 58us/step - loss: 0.4214 - acc: 0.8074 - val_loss: 0.4933 - val_acc: 0.7564
Epoch 5/10
20000/20000 [==============================] - 1s 59us/step - loss: 0.3981 - acc: 0.8205 - val_loss: 0.4905 - val_acc: 0.7576
Epoch 6/10
20000/20000 [==============================] - 1s 58us/step - loss: 0.3783 - acc: 0.8332 - val_loss: 0.4923 - val_acc: 0.7584
Epoch 7/10
20000/20000 [==============================] - 1s 60us/step - loss: 0.3606 - acc: 0.8429 - val_loss: 0.4957 - val_acc: